In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')

In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import os
import time
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

import h5py

from alvra_tools.load_data import *
from alvra_tools.channels import *

[INFO][corrections] Numba not available, reverting to Numpy


In [3]:
def get_xes(filename, DIR, DIRBS,roi):
    numstds = 2
    minIzero = 0.002
    image_threshold = .05
    hot_pixel = 7.5

#     filename = "Ru_foil_monoscan_001_step0010"

    print(DIR + filename + ".JF02T09V02crop.h5")
    print()
    print(DIRBS + filename + ".BSREAD.h5")
    
    images, _ = load_JF_cropped_data(DIR + filename + ".JF02T09V02crop.h5" , roi, nshots=None)



    (_, _, IzeroFEL, _, _, _, Energy, _, _, _ )\
        = load_PumpProbe_events_withTwoVariables(DIRBS + filename + ".BSREAD.h5", channel_BS_pulse_ids, channel_energy)

    total = 0


    IzeroMedian = np.median(IzeroFEL)
    IzeroSTD = np.std(IzeroFEL)

    nframes = images.shape

#     IzeroMedian = np.median(IzeroFEL)
#     IzeroSTD = np.std(IzeroFEL)

    print('number of frames')
    print(nframes)

    images_good = images

    images_thr = images_good.copy()
    images_thr[images_good < image_threshold] = 0
    images_thr[images_good > hot_pixel] = 0

    conditionMax = IzeroFEL < IzeroMedian+numstds*IzeroSTD
    conditionMin = IzeroFEL > IzeroMedian-numstds*IzeroSTD
    conditionLow = IzeroFEL > minIzero
    condition = np.logical_and.reduce((conditionLow, conditionMin, conditionMax)).T[0]

    images_thr = images_thr[condition]

    print('number of surviving frames')
    print(images_thr.shape[0])

    XES = images_thr.sum(axis=0)/images_thr.shape[0]

    print("XES size")
    print(XES.shape)

    return XES


In [4]:

#filename = "run_000407"
filename = "RuBpy3_monoscan_015_step0010"
#filename = "RuBpy3_monoscan_004_step0005"
# filename = "Ru_foil_XES_3keV_001"
#scan_name = "RuBpy3_monoscan_004"
scan_name = "2841eV_Rubpy3"
scan_name = "RuBpy3_monoscan_015"
DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
#DIR = "/das/work/p17/p17983/cropped_data/" + scan_name + "/"

DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"
#DIRBS = "/sf/alvra/data/p17983/raw/" + scan_name + "/"

XES_01 = get_xes(filename, DIR, DIRBS, 3)
XES_sum_01 = XES_01.sum(axis=0)

/das/work/p17/p17983/cropped_data/scan_data/RuBpy3_monoscan_015/RuBpy3_monoscan_015_step0010.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuBpy3_monoscan_015/RuBpy3_monoscan_015_step0010.BSREAD.h5
images_roi3
<KeysViewHDF5 ['coords_roi1', 'coords_roi2', 'coords_roi3', 'coords_roi4', 'images_roi1', 'images_roi2', 'images_roi3', 'images_roi4', 'pulse_ids']>
SLAAR11-LMOT-M453:ENC_1_BS/pulse_id
number of frames
(1000, 512, 300)


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1000 but corresponding boolean dimension is 500

In [ ]:
filename = "run_000382"
# scan_name = "Ru_foil_monoscan_001"
DIR = "/das/work/p17/p17983/cropped_data/Ru-foil/"#scan_data/" + scan_name + "/"
DIRBS = "/sf/alvra/data/p17983/raw/Ru-foil/"#scan_data/Ru_foil_monoscan_001/"
XES_02 = get_xes(filename, DIR, DIRBS)
XES_sum_02 = XES_02.sum(axis=0)

In [ ]:
plt.figure()
plt.imshow(XES_01, vmax = 0.1, vmin = 0)
plt.xlabel('JF pixel')
plt.ylabel('JF pixel')
plt.title('XES_PINK')
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(XES_02, vmax = 0.1, vmin = 0)
plt.xlabel('JF pixel')
plt.ylabel('JF pixel')
plt.title('XES_MONO')
plt.colorbar()

In [ ]:
plt.figure()
plt.plot(XES_sum_01,label="XES_PINK")
plt.plot(XES_sum_02,label="XES_MONO")
plt.plot(XES_sum_01-XES_sum_02,label="Diffence")
plt.xlabel('JF pixel')
plt.title('XES difference')
plt.xlabel('JF Pixel')
plt.ylabel('Counts of Some type')

plt.legend()

plt.xlim([0,200])